In [1]:
import os
import numpy as np
import glob
import random

import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

from visualize import *
from crowdnet_pytorch import CrowdNet
from hdf5_dataset import HDF5Dataset

In [2]:
model = CrowdNet(False)

In [3]:
print(model)

CrowdNet(
  (deep_network): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size

In [6]:
model_name = 'dcc_crowdnet'
model_path = os.path.expanduser(os.path.join('models', model_name))
data_path = os.path.expanduser(os.path.join('~/data', model_name))
weights_path = os.path.expanduser(os.path.join('~/models', model_name))

dataset_paths = 'dataset/UCF_CC_50'

slice_w = 256
slice_h = 256

patch_w = 225
patch_h = 225

net_density_h = 28
net_density_w = 28

In [13]:
for h5_file in glob.iglob(os.path.join(dataset_paths, '*.h5')):
    f = h5py.File(h5_file, 'r')

    # List all groups
    for key in f.keys():
        print(key)
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])
    print(i, len(data[0]), data[0][0], data[0][1])
    i += 1

density
150 640 0.0 0.0
density
151 600 0.00543679 0.00662574
density
152 896 0.000275478 0.000283342
density
153 1024 0.0 0.0
density
154 1024 0.0 0.0
density
155 1024 0.0 0.0
density
156 640 0.0 0.0
density
157 1024 0.00118483 0.00124133
density
158 1024 0.0 0.0
density
159 1024 0.0 0.0
density
160 1024 0.0 0.0
density
161 640 0.0 0.0
density
162 1024 4.78506e-07 5.27692e-07
density
163 1024 3.4773e-06 4.08374e-06
density
164 1024 0.0 0.0
density
165 1024 0.0 0.0
density
166 680 0.0 0.0
density
167 1024 0.000138662 0.000193009
density
168 640 0.0 0.0
density
169 600 0.000475062 0.000593323
density
170 1024 0.000876357 0.000920335
density
171 1024 7.6597e-05 9.77481e-05
density
172 1024 0.0 0.0
density
173 984 0.00386121 0.0046337
density
174 1024 0.00014224 0.00014617
density
175 1024 0.0 0.0
density
176 640 0.0 0.0
density
177 496 0.000195683 0.000220738
density
178 1024 0.000158383 0.000177716
density
179 1024 4.23852e-07 4.28305e-07
density
180 1024 2.07948e-05 2.13546e-05
density

In [ ]:
vis_square(list_to_np_array(X_train[:20]))

In [ ]:
#visualize one random density
i = int(random.random()*len(Y_train))
print (i)
print (np.sum(Y_train[i]))
plt.imshow(np.fliplr(Y_train[i]))